# Investigating factors related to quality in Portuguese Wines

In [ ]:
# import pandas as pd
from ucimlrepo import fetch_ucirepo


wine_quality = fetch_ucirepo(id=186) # This line may hang for a bit, it's a ~80mb data set
features = wine_quality.data.features.copy()
targets = wine_quality.data.targets.squeeze("columns")